In [47]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')  # ensure headless
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import io, datetime

# ---------- Config ----------
CSV_FILE = 'ngo_students.csv'
OUTPUT_HTML = 'dashboard.html'
LATEST_THRESHOLD = 5  # min enrollments per channel to include in "Best"

# ---------- Load & prep data ----------
df = pd.read_csv(CSV_FILE)

# normalize booleans
for col in ['dropped_out', 'placed']:
    df[col] = df[col].astype(str).str.strip().str.lower().map({'true': True, 'false': False})

# parse dates
for col in ['enrollment_date', 'graduation_date', 'placement_date']:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# month keys
df['enroll_month'] = df['enrollment_date'].dt.to_period('M')
df['grad_month'] = df['graduation_date'].dt.to_period('M')

# monthly dropout (by enrollment month)
monthly_enroll = df.dropna(subset=['enroll_month']).groupby('enroll_month').agg(
    enrollments=('student_id', 'count'),
    dropouts=('dropped_out', lambda s: int(np.sum(s.fillna(False))))
)
monthly_enroll['dropout_rate'] = monthly_enroll['dropouts'] / monthly_enroll['enrollments']

# monthly placement (by graduation month)
monthly_grad = df.dropna(subset=['grad_month']).groupby('grad_month').agg(
    graduates=('student_id', 'count'),
    placed=('placed', lambda s: int(np.sum(s.fillna(False))))
)
monthly_grad['placement_rate'] = monthly_grad['placed'] / monthly_grad['graduates']

# latest month = most recent with enrollments
latest_month = monthly_enroll.index.max()
latest_row = monthly_enroll.loc[latest_month]
latest_dropout_rate = latest_row['dropout_rate']
latest_enrollments = int(latest_row['enrollments'])
latest_dropouts = int(latest_row['dropouts'])

if latest_month in monthly_grad.index:
    lr = monthly_grad.loc[latest_month]
    latest_placement_rate = lr['placement_rate']
    latest_graduates = int(lr['graduates'])
    latest_placed = int(lr['placed'])
else:
    latest_placement_rate = np.nan
    latest_graduates = 0
    latest_placed = 0

# channel stats for latest enrollment month
df_latest = df[df['enroll_month'] == latest_month].copy()
chan = df_latest.groupby('onboarding_channel').agg(
    enrolled=('student_id', 'count'),
    placed_later=('placed', lambda s: int(np.sum(s.fillna(False))))
)
chan['conversion_rate'] = chan['placed_later'] / chan['enrolled']
chan_all = chan.sort_values(['conversion_rate','enrolled'], ascending=[False, False])
chan_best = chan_all[chan_all['enrolled'] >= LATEST_THRESHOLD]

# ---------- helper to convert matplotlib figure to inline SVG ----------
def fig_to_svg(fig):
    buf = io.StringIO()
    fig.savefig(buf, format='svg', bbox_inches='tight')
    plt.close(fig)
    svg = buf.getvalue()
    # strip the XML declaration to avoid duplicate declarations when embedding
    svg = svg.split('\n', 1)[-1] if svg.lstrip().startswith('<?xml') else svg
    return svg

# ---------- charts as inline SVG ----------
# Dropout rate (by enrollment month)
me = monthly_enroll.sort_index()
fig1 = plt.figure(figsize=(8, 3))
plt.plot(me.index.astype(str), me['dropout_rate'] * 100, marker='o', color='#d9534f')
plt.gca().yaxis.set_major_formatter(PercentFormatter(100))
plt.xticks(rotation=45, ha='right')
plt.title('Monthly Dropout Rate (by enrollment month)')
plt.xlabel('Month'); plt.ylabel('Rate (%)'); plt.grid(True, linestyle='--', alpha=0.4)
plt.tight_layout()
svg_dropout = fig_to_svg(fig1)

# Placement rate (by graduation month)
mg = monthly_grad.sort_index()
fig2 = plt.figure(figsize=(8, 3))
plt.plot(mg.index.astype(str), mg['placement_rate'] * 100, marker='o', color='#5cb85c')
plt.gca().yaxis.set_major_formatter(PercentFormatter(100))
plt.xticks(rotation=45, ha='right')
plt.title('Monthly Placement Rate (by graduation month)')
plt.xlabel('Month'); plt.ylabel('Rate (%)'); plt.grid(True, linestyle='--', alpha=0.4)
plt.tight_layout()
svg_placement = fig_to_svg(fig2)

# ---------- HTML ----------
month_str = str(latest_month)
now_str = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')

def kpi_card(label, value, subtitle=None):
    sub = f"<div class='kpi-sub'>{subtitle}</div>" if subtitle else ""
    return f"""
    <div class='kpi'>
      <div class='kpi-label'>{label}</div>
      <div class='kpi-value'>{value}</div>
      {sub}
    </div>
    """

kpi_html = "".join([
    kpi_card('Enrollments', f"{latest_enrollments}", f"Month: {month_str}"),
    kpi_card('Dropouts', f"{latest_dropouts}", f"Rate: {latest_dropout_rate*100:.1f}%"),
    kpi_card('Graduates', f"{latest_graduates}", f"Month: {month_str}"),
    kpi_card('Placed', f"{latest_placed}",
             f"Placement rate: {'' if np.isnan(latest_placement_rate) else f'{latest_placement_rate*100:.1f}%'}"),
])

best_rows = "".join([
    f"<tr><td>{idx}</td><td>{row['enrolled']}</td><td>{row['placed_later']}</td>"
    f"<td>{row['conversion_rate']*100:.1f}%</td></tr>"
    for idx, row in chan_best.iterrows()
])

all_rows = "".join([
    f"<tr><td>{idx}</td><td>{row['enrolled']}</td><td>{row['placed_later']}</td>"
    f"<td>{row['conversion_rate']*100:.1f}%</td></tr>"
    for idx, row in chan_all.iterrows()
])

html = f"""
<!DOCTYPE html>
<html lang='en'>
<head>
<meta charset='utf-8'>
<meta name='viewport' content='width=device-width, initial-scale=1'>
<title>Magic Bus Training Outcomes Dashboard</title>
<style>
  body {{ font-family: Arial, Helvetica, sans-serif; margin: 0; background:#f7f8fb; color:#222; }}
  header {{ background:#1f3b73; color:#fff; padding: 16px 24px; }}
  header h1 {{ margin:0; font-size: 20px; }}
  header .sub {{ opacity:0.85; font-size: 12px; }}
  .container {{ padding: 20px 24px; }}
  .kpis {{ display:grid; grid-template-columns: repeat(auto-fit, minmax(320px, 1fr)); grid-gap: 16px; margin-bottom: 20px; }}
  .kpi {{ display: flex;
  flex-direction: column;
  justify-content: flex-start;   /* or center if you prefer */
  align-items: flex-start;
  background: #fff;
  border: 1px solid #e5e7ef;
  border-radius: 12px;
  padding: 18px 20px;            /* more padding */
  min-height: 110px;             /* ensure height for big text */
  box-shadow: 0 1px 2px rgba(0,0,0,0.04);

}}

  .kpi-label {{ 
font-size:18px;            /* was ~12px */
  line-height:1.2;
  color:#1f3b73;             /* same blue as number */
  font-weight:700;           /* bold like the number */
  letter-spacing:.25px;
  text-transform:uppercase;  /* keep the uppercase look */
  margin-bottom:4px;
}}
  .kpi-value {{ 
font-size:32px;
  font-weight:700;
  margin-top:2px;
  color:#1f3b73;
}}
  .kpi-sub {{ 
font-size:12px; 
  color:#6b778c; 
  margin-top:6px;
 }}
  .section-title {{ font-size:16px; font-weight:700; margin: 18px 0 8px; color:#1f3b73; }}
  .card {{ background:#fff; border:1px solid #e5e7ef; border-radius:10px; padding:14px; margin-bottom:16px; box-shadow:0 1px 2px rgba(0,0,0,0.04); }}
  table {{ width:100%; border-collapse: collapse; }}
  th, td {{ text-align:left; padding:8px; border-bottom:1px solid #eee; font-size:13px; }}
  th {{ background:#f2f4f8; color:#333; }}
  .grid-2 {{ display:grid; grid-template-columns: 1fr 1fr; grid-gap:12px; }}
  @media (max-width:900px) {{ .grid-2 {{ grid-template-columns: 1fr; }} }}
  .footer {{ font-size:12px; color:#667085; margin-top:24px; }}
  .chart svg {{ width:100%; height:auto; border-radius:8px; border:1px solid #eee; }}
</style>
</head>
<body>
<header>
  <h1>Magic Bus Training Outcomes Dashboard</h1>
  <div class='sub'>Latest month: {month_str} • Generated {now_str}</div>
</header>
<div class='container'>
  <div class='kpis'>
    {kpi_html}
  </div>

  <div class='card'>
    <div class='section-title'>Best onboarding channels (min {LATEST_THRESHOLD} enrollments, month {month_str})</div>
    <table>
      <thead>
        <tr><th>Channel</th><th>Enrolled</th><th>Placed later</th><th>Conversion</th></tr>
      </thead>
      <tbody>
        {best_rows if best_rows else '<tr><td colspan=4>No channels met the threshold.</td></tr>'}
      </tbody>
    </table>
  </div>

  <div class='card'>
    <div class='section-title'>All channels (month {month_str})</div>
    <table>
      <thead>
        <tr><th>Channel</th><th>Enrolled</th><th>Placed later</th><th>Conversion</th></tr>
      </thead>
      <tbody>
        {all_rows}
      </tbody>
    </table>
  </div>

  <div class='grid-2'>
    <div class='card chart'>
      <div class='section-title'>Monthly Dropout Rate (by enrollment month)</div>
      {svg_dropout}
    </div>
    <div class='card chart'>
      <div class='section-title'>Monthly Placement Rate (by graduation month)</div>
      {svg_placement}
    </div>
  </div>

  <div class='footer'>Definitions: Dropout rate = dropouts / enrollments in month. Placement rate = placed / graduates in month. Conversion = placed later / enrolled in the latest month.</div>
</div>
</body>
</html>
"""

with open(OUTPUT_HTML, 'w', encoding='utf-8') as f:
    f.write(html)

print(f"Saved {OUTPUT_HTML}")


Saved dashboard.html
